# Capstone Project - Battle of The Neighborhoods 
### Applied Data Science Capstone

## Notebook Contents
* Introduction 
* Business Problem
* Target Audience
* Data
* Methodology
* Analysis
* Results and Discussion
* Conclusion

### Introduction
As part of Applied Data Science Capstone, this project encompasses the core concepts learned in the Coursera-IBM Data Science Professional Certificate. In the given context, this project employed techniques like data acquisition, cleaning, visualization, and analysis. Modeling based on machine learning algorithms is also a key component of this project. 

### Business Problem
Toronto, the capital of the province of Ontario, is one of the major Canadian cities. Being a dynamic metropolis, it attracts people from multi-ethnic groups, consequently it is not only the most densely populated region of the Canada but also provides numerous growth opportunities. In this context, the food business is one of the promising sectors to invest, as being a multicultural city, Toronto offers wide range of cuisines. 
To invest in the food business, this project aims to identify the best venue to open a restaurant that offers continental food, not only to better serve the community but also provide an edge to flourish the business. 

### Target Audience
The targeting audiences of this project are entrepreneurs and community members. As from the best location/venue of the continental restaurant, the entrepreneurs can benefit in terms of more profit from their business. On the other side, the best location of the continental restaurant will also facilitate the community members. 

### Data
As Toronto being the intended city to open the continental-food restaurant, the postal code data of Toronto is acquired from Wikipedia (the web-link to the acquired data is provided as, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). The acquired data is transformed to pandas dataframe for further processing. The extracted dataframe consist of 3 columns namely, Postal Codes, Borough, and Neighborhood, which are further cleaned for further visualization and analysis. The geographical coordinates of each postal code are obtained from https://cocl.us/Geospatial_data. Moreover, Geocoder and Foursquare are also employed in this project for visualizing and pertaining the venue data for geographical location suitability.

### Methodology


In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [3]:
# Data from Wikipedia
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(url).text

In [4]:
soup=BeautifulSoup(source, 'xml')
table=soup.find('table')

In [5]:
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['PostalCode','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names)

In [6]:
# Searching all PostalCode, Borough,and Neighborhood 
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [7]:
# Data Cleaning
df1=df[df['Borough']!='Not assigned']

In [8]:
# Combining the Neighborhoods with same PostalCode
df2 = df1.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df2.reset_index(inplace=True)
# Replacing the name of the Neighborhoods which are 'Not assigned' with names of Borough
df2['Neighborhood'] = np.where(df2['Neighborhood'] == 'Not assigned',df2['Borough'], df2['Neighborhood'])
df2

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [9]:
df2.shape

(103, 3)

#### Getting geospaciallocation coordinates and asisgining it to the corresponding borough and neighborhoods

In [10]:
# Installing and Importing necessary libraries

!pip install folium==0.5.0
import requests
import random
import folium
from geopy.geocoders import Nominatim
from IPython.display import Image 
from IPython.core.display import HTML 
from pandas.io.json import json_normalize

print('Libraries installed and imported')

     |████████████████████████████████| 79 kB 3.3 MB/s eta 0:00:011
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=a7c5ee5e25489ffcbe51ed6ec3b41faa3de2f7ae8834fb4a18858c4880a516e5
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
Libraries installed and imported


In [42]:
coordinates=pd.read_csv('https://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
coordinates.rename(columns={'Postal Code':'PostalCode'},inplace=True)

In [44]:
coordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [45]:
df3=pd.merge(df2, coordinates, on='PostalCode')
df3

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Clustering the Neighborhoods in Toronto

#### Geospacial data visualization

In [53]:
map_toronto = folium.Map(location=[43.6534817, -79.3839347],zoom_start=10)
for lat,lng,borough,neighborhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)
map_toronto

### Link for Viewing Maps
##### If the map is not visible in Github, kindly click the link given below,

https://nbviewer.jupyter.org/github/attique87/Coursera_Capstone/blob/main/Final%20Capstone%20Project.ipynb

### Foursquare API

In [54]:
CLIENT_ID = 'YA4GZAU5NUZ1T1F4DH5U4KKMFDCIUR3RNGAJUC230CRSRQXE'
CLIENT_SECRET = '022HUWS3PZMTAFI2SEK3ARL1DSPAUXGV351FBBZP4S22BC1U'
ACCESS_TOKEN = 'IMPAE0NW5GZGOY4IVLBF0TFVSTRLUE0ET3MZDJSOLC5BGGLN'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YA4GZAU5NUZ1T1F4DH5U4KKMFDCIUR3RNGAJUC230CRSRQXE
CLIENT_SECRET:022HUWS3PZMTAFI2SEK3ARL1DSPAUXGV351FBBZP4S22BC1U


#### Defining an instance of the geocoder

In [55]:
address = 'Toronto'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [89]:
search_query = 'Continental Food'
radius = 100000
print(search_query + ' .... OK!')

Continental Food .... OK!


In [90]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=YA4GZAU5NUZ1T1F4DH5U4KKMFDCIUR3RNGAJUC230CRSRQXE&client_secret=022HUWS3PZMTAFI2SEK3ARL1DSPAUXGV351FBBZP4S22BC1U&ll=43.6534817,-79.3839347&oauth_token=IMPAE0NW5GZGOY4IVLBF0TFVSTRLUE0ET3MZDJSOLC5BGGLN&v=20180604&query=Continental Food&radius=100000&limit=30'

In [91]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fd711eeea3ff877cf8c4e7e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4c8c4879cf3ea1436a3cf451',
    'name': 'Continental',
    'location': {'address': '15 DunDonald St.',
     'crossStreet': 'Yonge',
     'lat': 43.665513246213614,
     'lng': -79.38415668564011,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.665513246213614,
       'lng': -79.38415668564011}],
     'distance': 1339,
     'postalCode': 'M4Y',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['15 DunDonald St. (Yonge)',
      'Toronto ON M4Y',
      'Canada']},
    'categories': [],
    'referralId': 'v-1607930351',
    'hasPerk': False},
   {'id': '50f2064de4b0403bd6e3ddc1',
    'name': 'World Food Market',
    'location': {'address': '335 Yonge Street',
     'crossStreet': 'Gould Street',
     'lat': 43.65737802426023,
     'lng': -79.3

In [92]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.neighborhood,venuePage.id
0,4c8c4879cf3ea1436a3cf451,Continental,[],v-1607930351,False,15 DunDonald St.,Yonge,43.665513,-79.384157,"[{'label': 'display', 'lat': 43.66551324621361...",1339,M4Y,CA,Toronto,ON,Canada,"[15 DunDonald St. (Yonge), Toronto ON M4Y, Can...",NaN,NaN
1,50f2064de4b0403bd6e3ddc1,World Food Market,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",v-1607930351,False,335 Yonge Street,Gould Street,43.657378,-79.381321,"[{'label': 'display', 'lat': 43.65737802426023...",482,M5B 2L3,CA,Toronto,ON,Canada,"[335 Yonge Street (Gould Street), Toronto ON M...",Downtown Toronto,NaN
2,4cbc732b035d236ac1a2e54e,Saha Mediterranean Fast Food,"[{'id': '4bf58dd8d48988d1c0941735', 'name': 'M...",v-1607930351,False,81 Gerrard St W,La Plante Ave,43.658497,-79.386035,"[{'label': 'display', 'lat': 43.658497, 'lng':...",583,M7A 2H4,CA,Toronto,ON,Canada,"[81 Gerrard St W (La Plante Ave), Toronto ON M...",NaN,NaN
3,4f6b4904e4b0d8154bf53aa5,10 Dundas East Food Court,"[{'id': '4bf58dd8d48988d120951735', 'name': 'F...",v-1607930351,False,10 Dundas St. E,Yonge St.,43.656252,-79.380899,"[{'label': 'display', 'lat': 43.65625212347179...",393,M7A 2A1,CA,Toronto,ON,Canada,"[10 Dundas St. E (Yonge St.), Toronto ON M7A 2...",NaN,NaN
4,5f51173513c5c74d1b83cd09,Tut's Egyptian Street Food,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1607930351,False,567 King St W,Portland,43.644660,-79.399470,"[{'label': 'display', 'lat': 43.64466, 'lng': ...",1590,M5V 1M1,CA,Toronto,ON,Canada,"[567 King St W (Portland), Toronto ON M5V 1M1,...",Fashion District,NaN


In [93]:
dataframe.shape

(30, 19)

#### Filtering the extracted results

In [94]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Continental,None,15 DunDonald St.,Yonge,43.665513,-79.384157,"[{'label': 'display', 'lat': 43.66551324621361...",1339,M4Y,CA,Toronto,ON,Canada,"[15 DunDonald St. (Yonge), Toronto ON M4Y, Can...",NaN,4c8c4879cf3ea1436a3cf451
1,World Food Market,Food Truck,335 Yonge Street,Gould Street,43.657378,-79.381321,"[{'label': 'display', 'lat': 43.65737802426023...",482,M5B 2L3,CA,Toronto,ON,Canada,"[335 Yonge Street (Gould Street), Toronto ON M...",Downtown Toronto,50f2064de4b0403bd6e3ddc1
2,Saha Mediterranean Fast Food,Mediterranean Restaurant,81 Gerrard St W,La Plante Ave,43.658497,-79.386035,"[{'label': 'display', 'lat': 43.658497, 'lng':...",583,M7A 2H4,CA,Toronto,ON,Canada,"[81 Gerrard St W (La Plante Ave), Toronto ON M...",NaN,4cbc732b035d236ac1a2e54e
3,10 Dundas East Food Court,Food Court,10 Dundas St. E,Yonge St.,43.656252,-79.380899,"[{'label': 'display', 'lat': 43.65625212347179...",393,M7A 2A1,CA,Toronto,ON,Canada,"[10 Dundas St. E (Yonge St.), Toronto ON M7A 2...",NaN,4f6b4904e4b0d8154bf53aa5
4,Tut's Egyptian Street Food,Fast Food Restaurant,567 King St W,Portland,43.644660,-79.399470,"[{'label': 'display', 'lat': 43.64466, 'lng': ...",1590,M5V 1M1,CA,Toronto,ON,Canada,"[567 King St W (Portland), Toronto ON M5V 1M1,...",Fashion District,5f51173513c5c74d1b83cd09
5,JC Food Market,Grocery Store,140 Simcoe St,btwn Richmond St W & Nelson St,43.649212,-79.386916,"[{'label': 'display', 'lat': 43.64921175406505...",532,NaN,CA,Toronto,ON,Canada,[140 Simcoe St (btwn Richmond St W & Nelson St...,NaN,4c9bc2ed0e9bb1f7f8a7cd5f
6,Classic italian style pizza food truck,Food Truck,CNE Midway,NaN,43.652144,-79.381118,"[{'label': 'display', 'lat': 43.652144, 'lng':...",271,NaN,CA,Toronto,ON,Canada,"[CNE Midway, Toronto ON, Canada]",NaN,4c787c9181bca0936180fa14
7,Sam's Food Stores,Deli / Bodega,College,Augusta,43.657317,-79.402995,"[{'label': 'display', 'lat': 43.65731726464263...",1593,NaN,CA,Toronto,ON,Canada,"[College (Augusta), Toronto ON, Canada]",NaN,4b79bcd0f964a520b90e2fe3
8,Atrium On Bay Food Court,Food Court,595 Bay St.,at Dundas St. W,43.656169,-79.383352,"[{'label': 'display', 'lat': 43.65616874431004...",302,M5G 2N8,CA,Toronto,ON,Canada,"[595 Bay St. (at Dundas St. W), Toronto ON M5G...",NaN,4f147d1ce4b03f4c02b6f273
9,30 Minutes or Less Food Truck,Food Truck,NaN,NaN,43.656423,-79.379897,"[{'label': 'display', 'lat': 43.65642348437276...",461,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4e26052e315161d79d2a8203


#### Name of restaurant offering continental food in the selcted region

In [95]:
dataframe_filtered.name

0                                Continental
1                          World Food Market
2               Saha Mediterranean Fast Food
3                  10 Dundas East Food Court
4                 Tut's Egyptian Street Food
5                             JC Food Market
6     Classic italian style pizza food truck
7                          Sam's Food Stores
8                   Atrium On Bay Food Court
9              30 Minutes or Less Food Truck
10            Nature's Alternate Health Food
11              金城超級市場 Lucky Moose Food Mart
12                          Fine Food Market
13                    Food Cabbie Food Truck
14        InterContinental Toronto Yorkville
15                                Food Court
16              Shops at Sheraton Food Court
17                  Local Toronto Food Tours
18                   Green Republic Food Co.
19                     Toronto Food Cart Lot
20                        Ying Ying Soy Food
21                          Fine Food on Bay
22        

#### Visualizing the restaurants, offering continental food, on the interactive map

In [116]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='black',
        popup=label,
        fill = True,
        fill_color='black',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Link for Viewing Maps
##### If the map is not visible in Github, kindly click the link given below,

https://nbviewer.jupyter.org/github/attique87/Coursera_Capstone/blob/main/Final%20Capstone%20Project.ipynb

In [117]:
map_toronto = folium.Map(location=[43.6534817, -79.3839347],zoom_start=10)
for lat,lng,borough,neighborhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius=5, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.7, parse_html=False).add_to(map_toronto)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker([lat, lng], radius=5, color='black', popup=label, fill = True, fill_color='black', fill_opacity=0.6).add_to(map_toronto)

# display map
map_toronto

### Link for Viewing Maps
##### If the map is not visible in Github, kindly click the link given below,

https://nbviewer.jupyter.org/github/attique87/Coursera_Capstone/blob/main/Final%20Capstone%20Project.ipynb

### Clustering

In [125]:
from sklearn.cluster import KMeans # Importing KMeans
k=5 # Cluster Size of 5
toronto_cluster=df3.drop(['PostalCode','Borough','Neighborhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_cluster)
kmeans.labels_
df3.insert(0, 'Cluster Labels', kmeans.labels_)

In [126]:
# Dataframe with Cluster Labels
df3

,Cluster Labels,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...,...
98,1,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,2,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,4,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,1,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Geospacial Visualization of Clustering of Neighborhoods

In [128]:
# Importing necessary libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

# Creating Map
map_clusters = folium.Map(location=[43.6534817, -79.3839347],zoom_start=10)

# Setting Color Scheme for Clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Adding Markers to Map
markers_colors = []
for lat, lon, neighburhood, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood'], df3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

### Link for Viewing Maps
##### If the map is not visible in Github, kindly click the link given below,

https://nbviewer.jupyter.org/github/attique87/Coursera_Capstone/blob/main/Final%20Capstone%20Project.ipynb

In [129]:
# Importing necessary libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

# Creating Map
map_clusters = folium.Map(location=[43.6534817, -79.3839347],zoom_start=10)

# Setting Color Scheme for Clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Adding Markers to Map
markers_colors = []
for lat, lon, neighburhood, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighborhood'], df3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)


for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker([lat, lng], radius=5, color='black', popup=label, fill = True, fill_color='black', fill_opacity=0.6).add_to(map_clusters)

# display map
map_clusters

### Link for Viewing Maps
##### If the map is not visible in Github, kindly click the link given below,

https://nbviewer.jupyter.org/github/attique87/Coursera_Capstone/blob/main/Final%20Capstone%20Project.ipynb